In [79]:
from langchain.utilities import SQLDatabase
from langchain.agents import create_sql_agent  
from langchain.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_google_genai import GoogleGenerativeAI  

api_key = "My_API_Key"

llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=api_key, temperature=0.2)

db_user="root"
db_password="2112"
db_host="localhost"
db_name="atliq_tshirts"
db=SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)



In [80]:
db_agent=create_sql_agent(llm,db=db,verbose=True)

In [81]:

qns1=db_agent.run("How many t-shirts do we  have left for nike in extra small size and white color?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
The t_shirts table seems relevant._shirts
Action: sql_db_schema
Action Input: t_shirts
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	S	13	72
2	Van Huesen	Black	L	11	78
3	Nike	Blue	XS	12	84
 I need to query the t_shirts table for the rows where the brand is 'Nike', the color is 'White', and the size is 'XS'.  I should then sum the stock_quantity column to get the total number of t-shirts in stock.
Action: sql_db_query_checker
SELECT SUM(stoc

In [82]:
qns1

'69'

In [83]:
qns2=db_agent.run("Hoe much price in the inventory for all small size t-dhirts?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
I should query the schema of the t_shirts table.
Action: sql_db_schema
Action Input: t_shirts
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	S	13	72
2	Van Huesen	Black	L	11	78
3	Nike	Blue	XS	12	84
 I should query the t_shirts table for all rows where the size is 'S' and sum the price column.
Action: sql_db_query_checker
SELECT SUM(price) FROM t_shirts WHERE size = 'S';Action: sql_db_query
Final Answer: 505OM t_shirts WHERE size = 'S';[(Decim

In [84]:
qns2=db_agent.run("SELECT SUM(PRICE*STOCK_QUANTITY) FROM t_shirts WHERE SIZE='S'")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
The t_shirts table looks relevant.  I should query the schema of this table.
Action: sql_db_schema
Action Input: t_shirts
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	S	13	72
2	Van Huesen	Black	L	11	78
3	Nike	Blue	XS	12	84
The t_shirts table has the columns I need to answer the question.  I should construct a query to get the sum of the price times the stock quantity for all t-shirts where the size is S.
Action: sql_db_query_checker
SELECT

In [85]:
qns2

'24137'

In [86]:
qns3=db_agent.run("if we have sell all Levi's t-shirts today with discounts applied.How much revenue our store will generate (post discounts)?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
I should query the schema of the t_shirts table to see what columns are available.
Action: sql_db_schema
Action Input: t_shirts
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	S	13	72
2	Van Huesen	Black	L	11	78
3	Nike	Blue	XS	12	84
I should query the schema of the discounts table to see what columns are available.
Action: sql_db_schema
Action Input: discounts
CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id 

In [87]:
qns3=db_agent.run('''SELECT sum(a.total_amount*((100-coalesce(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand='Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id=discounts.t_shirt_id''')



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
I should query the schema of the t_shirts table.
Action: sql_db_schema
Action Input: t_shirts
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	S	13	72
2	Van Huesen	Black	L	11	78
3	Nike	Blue	XS	12	84
I should query the schema of the discounts table.
Action: sql_db_schema
Action Input: discounts
CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (disco

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Final Answer: 884

> Finished chain.


In [88]:
qns3

'884'

In [89]:
qns4=db_agent.run("How many white color levi's t-shirts are available?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
I should query the schema of the t_shirts table.
Action: sql_db_schema
Action Input: t_shirts
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	S	13	72
2	Van Huesen	Black	L	11	78
3	Nike	Blue	XS	12	84
 I should query the t_shirts table for the number of white color levi's t-shirts.
Action: sql_db_query_checker
SELECT COUNT(*) FROM t_shirts WHERE brand = 'Levi' AND color = 'White';Action: sql_db_query
Final Answer: 5OM t_shirts WHERE brand = 'Lev

In [90]:
qns4=db_agent.run("SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Levi' AND color='White';")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
I should query the schema of the t_shirts table.
Action: sql_db_schema
Action Input: t_shirts
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	S	13	72
2	Van Huesen	Black	L	11	78
3	Nike	Blue	XS	12	84
I should query the t_shirts table to get the sum of stock_quantity for Levi brand and White color.
Action: sql_db_query_checker
SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Levi' AND color='White';Final Answer: 10

> Finished chain.


In [91]:
qns4

'10'

In [92]:
qns5=db_agent.run("if we have sell all Levi's t-shirts.How much revenue our store will generate (post discounts)?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
I should query the schema of the t_shirts table.
Action: sql_db_schema
Action Input: t_shirts
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	S	13	72
2	Van Huesen	Black	L	11	78
3	Nike	Blue	XS	12	84
I should query the schema of the discounts table.
Action: sql_db_schema
Action Input: discounts
CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (disco

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


I should query the t_shirts and discounts tables to find the total revenue from selling all Levi's t-shirts.
Action: sql_db_query_checker
Action Input: SELECT SUM(t_shirts.price * t_shirts.stock_quantity * (1 - discounts.pct_discount)) AS total_revenue
FROM t_shirts
JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id
SELECT SUM(t_shirts.price * t_shirts.stock_quantity * (1 - discounts.pct_discount)) AS total_revenue
FROM t_shirts
JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id
Action: sql_db_querym
Action Input: SELECT SUM(t_shirts.price * t_shirts.stock_quantity * (1 - discounts.pct_discount)) AS total_revenue
FROM t_shirts
JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id
Final Answer: -842436;1m[(Decimal('-8424.00'),)]

> Finished chain.


  Few Shot Learning

In [93]:
few_shots=[
    {
        "Question" : "How many t-shirts do we  have left for nike in extra small size and white color?",
        "SQLQuery" : "SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS';",
        "SQLResult" : "Result of the SQL Query",
        "Answer"   : qns1},
    {
        "Question" : "How much price in the inventory for all small size t-shirts?",
        "SQLQuery" : "SELECT SUM(PRICE*STOCK_QUANTITY) FROM t_shirts WHERE SIZE='S'",
        "SQLResult" : "Result of the SQL Query",
        "Answer" : qns2},
    {
        "Question" : "if we have sell all Levi's t-shirts today with discounts applied.How much revenue our store will generate (post discounts)?",
        "SQLQuery" : """SELECT sum(a.total_amount*((100-coalesce(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand='Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id=discounts.t_shirt_id;""",
        "SQLResult" : "Result of the SQL Query",
        "Answer" : qns3},
    {
        "Question" : "How many white color levi's t-shirts are available?",
        "SQLQuery" : "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Levi' AND color='White';",
        "SQLResult" : "Result of the SQL Query",
        "Answer" : qns4},
    {
        "Question" : "if we have sell all Levi's t-shirts.How much revenue our store will generate (post discounts)?",
        "SQLQuery" : "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
        "SQLResult" : "Result of the SQL Query",
        "Answer" : qns5}
]

In [94]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]


In [95]:
to_vectorize

["How many t-shirts do we  have left for nike in extra small size and white color? SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'; Result of the SQL Query 69",
 "How much price in the inventory for all small size t-shirts? SELECT SUM(PRICE*STOCK_QUANTITY) FROM t_shirts WHERE SIZE='S' Result of the SQL Query 24137",
 "if we have sell all Levi's t-shirts today with discounts applied.How much revenue our store will generate (post discounts)? SELECT sum(a.total_amount*((100-coalesce(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand='Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id=discounts.t_shirt_id; Result of the SQL Query 884",
 "How many white color levi's t-shirts are available? SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Levi' AND color='White'; Result of the SQL Query 10",
 "if we have sell all Levi's t-shirts.How muc

In [96]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [97]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '69',
  'Question': 'How many t-shirts do we  have left for nike in extra small size and white color?',
  'SQLQuery': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS';",
  'SQLResult': 'Result of the SQL Query'},
 {'Answer': '69',
  'Question': 'How many t-shirts do we  have left for nike in extra small size and white color?',
  'SQLQuery': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS';",
  'SQLResult': 'Result of the SQL Query'}]

In [98]:
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [99]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [100]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [101]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [102]:
from langchain_experimental.sql import SQLDatabaseChain

In [105]:
new_agent = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [106]:
new_agent("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Levi' AND color='White';
SQLResult: [(Decimal('293'),)]
293
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '293'}

In [107]:
new_agent("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SELECT SUM(PRICE*STOCK_QUANTITY) FROM t_shirts WHERE SIZE='S'
SQLResult: [(Decimal('24137'),)]
24137
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': '24137'}

In [108]:
new_agent("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SELECT sum(a.total_amount*((100-coalesce(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand='Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id=discounts.t_shirt_id;
SQLResult: [(Decimal('27240.400000'),)]
27240.40
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '27240.40'}

In [109]:
new_agent.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SELECT sum(price*stock_quantity) as total_revenue from t_shirts where brand='Van Huesen';
SQLResult: [(Decimal('31290'),)]
31290
> Finished chain.


'31290'